### 生成测试数据

* 10组每组10张 训练数据
    * img_i
    * 0~10.png
    * trainData_i.dt
* 1组10张 测试数据
    * imgTest
    * testData

In [ ]:
# mkdir
import os,sys
path = "./testDir"
if os.path.exists(path):
    print('dir exist')
else:
    os.makedirs(path)

In [ ]:
import numpy as np
import cv2 as cv
import random
import os,sys

imgW = 28
imgH = 28

def generalRec():
    img = np.ones((imgW,imgH),dtype=np.uint8)
    InitX = random.randint(2,imgW-5)
    InitY = random.randint(2,imgH-5)
    RecW = random.randint(4,imgW-InitX-1)
    RecH = random.randint(4,imgH-InitY-1)
    cv.rectangle(img,(InitX,InitY),(RecW,RecH),255,1)
#     #显示图片
#     cv.namedWindow("img")
#     cv.imshow("img",img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    return img


    
def generalCircle():
    img = np.ones((imgW,imgH),dtype=np.uint8)
    InitX = random.randint(5,imgW-5)
    InitY = random.randint(5,imgH-5)
    minDis = min(5,imgW-InitX,imgH-InitY)
    r = random.randint(2,minDis)
    cv.circle(img,(InitX,InitY),r,255,1)
#     #显示图片
#     cv.namedWindow("img")
#     cv.imshow("img",img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    return img
    

def GenerateTrainData(genNum,imageDir,dataFilePath):
#     trainDataFile = "./trainData/trainData_"+str(index)+".dt" 
    with open(dataFilePath,'w') as f:
        for i in range(genNum):
            randBoo = random.choice([True,False])
            if(randBoo):
                # general rec
                genImg = generalRec()
                lineStr = imageDir+"/"+i.__str__()+".png,0\n"
                f.write(lineStr)
                cv.imwrite(imageDir+"/"+i.__str__()+".png",genImg)
            else:
                # general circle
                genImg = generalCircle()
                lineStr = imageDir+"/"+i.__str__()+".png,1\n"
                f.write(lineStr)
                cv.imwrite(imageDir+"/"+i.__str__()+".png",genImg)

                
                
                
dataDir = "./trainData"
if os.path.exists(dataDir):
    print('DataDir exist')
else:
    os.makedirs(dataDir)    

for i in range(10):
    path = "./img/img_"+str(i)
    if os.path.exists(path):
        print('dir exist')
    else:
        os.makedirs(path)
    dataFilePath = "./trainData/trainData_"+str(i)+".dt" 
    GenerateTrainData(10,path,dataFilePath)
    
testImgPath = "./img/img_test"
if os.path.exists(testImgPath):
    print('DataDir exist')
else:
    os.makedirs(testImgPath)    
testDataFilePath = "./trainData/testData.dt" 
GenerateTrainData(10,testImgPath,testDataFilePath)
    


### 数据读取验证

In [ ]:
    line = f.readline()
    while line:
        strSplit=line.split(',',1)
        label_input=int(strSplit[1])
        print(label_input)
        img = cv.imread(strSplit[0],cv.IMREAD_GRAYSCALE)
#         print(img)
#         npImg = np.reshape(img,(1,784))
#         print(npImg)
#         cv.namedWindow("img")
#         cv.imshow("img",img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()
        line = f.readline()


In [ ]:
import cv2 as cv
import numpy

def readCustomDataFromFile(trainDataPath):
    read_img = np.empty((10,1,28,28),dtype='float32')
    read_label = np.empty((10,1),dtype='int64')

    with open(trainDataPath) as f:
        for i in range(10):
            line = f.readline()
            if line:
                strSplit=line.split(',',1)
                img = cv.imread(strSplit[0],cv.IMREAD_GRAYSCALE)
                read_img[i][0] = img
                read_label[i] = int(strSplit[1])
            else:
                break
    print(read_label)
    read_img =read_img/255.0*2.0 - 1.0
    return read_img,read_label



a,b = readCustomDataFromFile('trainData/trainData_1.dt')

# http://www.paddlepaddle.org/documentation/docs/zh/1.2/user_guides/howto/prepare_data/reader_cn.html
# images = images / 255.0 * 2.0 - 1.0


In [ ]:
import numpy as np

a = np.empty((32,1),dtype=float)
a[0][0]=123
# print(a)

b =np.empty((1,1,2,2),dtype=float)
c =np.empty((2,2),dtype=float)
c[0][0] = 1.1
c[0][1] = 2.2
c[1][0] = 3.3
c[1][1] = 4.4
b[0][0] = c
print(c)
print(b)


In [ ]:
a=numpy.random.random(size=(10, 1, 28, 28)).astype('float32')
print(a)
b=numpy.random.random(size=(10, 1)).astype('int64')
print(b)

### 训练

In [1]:
from __future__ import print_function

import os
from PIL import Image
import numpy
import paddle
import paddle.fluid as fluid
import cv2 as cv

BATCH_SIZE = 64
PASS_NUM = 5


def readCustomDataFromFile(trainDataPath):
    read_img = numpy.empty((10,1,28,28),dtype='float32')
    read_label = numpy.empty((10,1),dtype='int64')

    with open(trainDataPath) as f:
        for i in range(10):
            line = f.readline()
            if line:
                strSplit=line.split(',',1)
                img = cv.imread(strSplit[0],cv.IMREAD_GRAYSCALE)
                read_img[i][0] = img
                read_label[i] = int(strSplit[1])
            else:
                break
    read_img =read_img/255.0*2.0 - 1.0
    return read_img,read_label




def loss_net(hidden, label):
    prediction = fluid.layers.fc(input=hidden, size=10, act='softmax')
    loss = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_loss = fluid.layers.mean(loss)
    acc = fluid.layers.accuracy(input=prediction, label=label)
    return prediction, avg_loss, acc


def multilayer_perceptron(img, label):
    img = fluid.layers.fc(input=img, size=200, act='tanh')
    hidden = fluid.layers.fc(input=img, size=200, act='tanh')
    return loss_net(hidden, label)


def softmax_regression(img, label):
    return loss_net(img, label)


def convolutional_neural_network(img, label):
    conv_pool_1 = fluid.nets.simple_img_conv_pool(
        input=img,
        filter_size=5,
        num_filters=20,
        pool_size=2,
        pool_stride=2,
        act="relu")
    conv_pool_1 = fluid.layers.batch_norm(conv_pool_1)
    conv_pool_2 = fluid.nets.simple_img_conv_pool(
        input=conv_pool_1,
        filter_size=5,
        num_filters=50,
        pool_size=2,
        pool_stride=2,
        act="relu")
    return loss_net(conv_pool_2, label)


def train(nn_type,
          use_cuda,
          save_dirname=None,
          model_filename=None,
          params_filename=None):
    if use_cuda and not fluid.core.is_compiled_with_cuda():
        return

    img = fluid.layers.data(name='img', shape=[1, 28, 28], dtype='float32')
    label = fluid.layers.data(name='label', shape=[1], dtype='int64')

    if nn_type == 'softmax_regression':
        net_conf = softmax_regression
    elif nn_type == 'multilayer_perceptron':
        net_conf = multilayer_perceptron
    else:
        net_conf = convolutional_neural_network

    prediction, avg_loss, acc = net_conf(img, label)

    test_program = fluid.default_main_program().clone(for_test=True)

    optimizer = fluid.optimizer.Adam(learning_rate=0.001)
    optimizer.minimize(avg_loss)

    def train_test(train_test_program):
        acc_set = []
        avg_loss_set = []
        readImg,readLabel = readCustomDataFromFile('trainData/testData.dt')
        acc_np, avg_loss_np = exe.run(
            test_program,
            feed={
                "img": readImg,
                "label": readLabel
            },
            fetch_list=[acc, avg_loss])
        acc_set.append(float(acc_np))
        avg_loss_set.append(float(avg_loss_np))
        # get test acc and loss
        acc_val_mean = numpy.array(acc_set).mean()
        avg_loss_val_mean = numpy.array(avg_loss_set).mean()
        return avg_loss_val_mean, acc_val_mean

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

    exe = fluid.Executor(place)

    train_reader = paddle.batch(
        paddle.reader.shuffle(paddle.dataset.mnist.train(), buf_size=500),
        batch_size=BATCH_SIZE)
    test_reader = paddle.batch(
        paddle.dataset.mnist.test(), batch_size=BATCH_SIZE)
    feeder = fluid.DataFeeder(feed_list=[img, label], place=place)

    exe.run(fluid.default_startup_program())
    main_program = fluid.default_main_program()
    epochs = [epoch_id for epoch_id in range(PASS_NUM)]

    lists = []

#     for epoch_id in epochs:
    for epoch_id in range(10):
        trainDataFilePath = 'trainData/trainData_'+str(epoch_id)+'.dt'
        readImg,readLabel = readCustomDataFromFile(trainDataFilePath)
        metrics = exe.run(
            main_program,
            feed={
                "img": readImg,
                "label": readLabel
            },
            fetch_list=[avg_loss, acc])

        print(" Batch %d, Cost %f" % (epoch_id,metrics[0]))

            
        # test for epoch
        avg_loss_val, acc_val = train_test(test_program)
        print("Test with Epoch %d, avg_cost: %s, acc: %s" %
              (epoch_id, avg_loss_val, acc_val))
        lists.append((epoch_id, avg_loss_val, acc_val))
        if save_dirname is not None:
            fluid.io.save_inference_model(
                save_dirname, ["img"], [prediction],
                exe,
                model_filename=model_filename,
                params_filename=params_filename)

    # find the best pass
#     best = sorted(lists, key=lambda list: float(list[1]))[0]
#     print('Best pass is %s, testing Avgcost is %s' % (best[0], best[1]))
#     print('The classification accuracy is %.2f%%' % (float(best[2]) * 100))


def infer(use_cuda,
          save_dirname=None,
          model_filename=None,
          params_filename=None):
    if save_dirname is None:
        return

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    exe = fluid.Executor(place)

#     def load_image(file):
#         im = Image.open(file).convert('L')
#         im = im.resize((28, 28), Image.ANTIALIAS)
#         im = numpy.array(im).reshape(1, 1, 28, 28).astype(numpy.float32)
#         im = im / 255.0 * 2.0 - 1.0
#         return im

    def load_image(file):
        read_img = numpy.empty((1,1,28,28),dtype='float32')
        im = cv.imread(file,cv.IMREAD_GRAYSCALE)
        read_img[0][0] = im
        read_img =read_img/255.0*2.0 - 1.0
        return read_img
    
    tensor_img = load_image('/image/rec.png')
#     tensor_img = load_image('/image/circle.png')
    

    inference_scope = fluid.core.Scope()
    with fluid.scope_guard(inference_scope):
        # Use fluid.io.load_inference_model to obtain the inference program desc,
        # the feed_target_names (the names of variables that will be feeded
        # data using feed operators), and the fetch_targets (variables that
        # we want to obtain data from using fetch operators).
        [inference_program, feed_target_names,
         fetch_targets] = fluid.io.load_inference_model(
             save_dirname, exe, model_filename, params_filename)

        # Construct feed as a dictionary of {feed_target_name: feed_target_data}
        # and results will contain a list of data corresponding to fetch_targets.
        results = exe.run(
            inference_program,
            feed={feed_target_names[0]: tensor_img},
            fetch_list=fetch_targets)
        lab = numpy.argsort(results)
        print("Inference result of image/rec.png is: %d" % lab[0][0][-1])


def main(use_cuda, nn_type):
    model_filename = None
    params_filename = None
    save_dirname = "recognize_digits_" + nn_type + ".inference.model"

    # call train() with is_local argument to run distributed train
    train(
        nn_type=nn_type,
        use_cuda=use_cuda,
        save_dirname=save_dirname,
        model_filename=model_filename,
        params_filename=params_filename)
    print('train finsh~~~')
    print(save_dirname)
    print(model_filename)
    infer(
        use_cuda=use_cuda,
        save_dirname=save_dirname,
        model_filename=model_filename,
        params_filename=params_filename)


if __name__ == '__main__':
    use_cuda = True
    #predict = 'softmax_regression' # uncomment for Softmax
    #predict = 'multilayer_perceptron' # uncomment for MLP
    predict = 'convolutional_neural_network'  # uncomment for LeNet5
    main(use_cuda=use_cuda, nn_type=predict)

 Batch 0, Cost 2.413728
Test with Epoch 0, avg_cost: 1.2064870595932007, acc: 0.30000001192092896
 Batch 1, Cost 1.024712
Test with Epoch 1, avg_cost: 0.9445624351501465, acc: 0.5
 Batch 2, Cost 1.214199
Test with Epoch 2, avg_cost: 0.8885771036148071, acc: 0.4000000059604645
 Batch 3, Cost 1.424419
Test with Epoch 3, avg_cost: 0.683829128742218, acc: 0.699999988079071
 Batch 4, Cost 0.330427
Test with Epoch 4, avg_cost: 0.5954464077949524, acc: 0.699999988079071
 Batch 5, Cost 0.530659
Test with Epoch 5, avg_cost: 0.5131652355194092, acc: 0.800000011920929
 Batch 6, Cost 1.040055
Test with Epoch 6, avg_cost: 0.4793611466884613, acc: 0.9000000357627869
 Batch 7, Cost 0.600359
Test with Epoch 7, avg_cost: 0.482517808675766, acc: 0.800000011920929
 Batch 8, Cost 2.097363
Test with Epoch 8, avg_cost: 0.5515785217285156, acc: 0.800000011920929
 Batch 9, Cost 0.398899
Test with Epoch 9, avg_cost: 0.6600465178489685, acc: 0.699999988079071
train finsh~~~
recognize_digits_convolutional_neural

In [ ]:
### 记录
* 运行第二次的时候会报错
    * Restart Kernel